In [1]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say current_sub state /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

In [2]:
my $practice_input = qq/
Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0
/;


Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0


In [213]:
sub process_day17_part1 {
    my ($input) = @_;
    my ($registers, $program) = split "\n\n", $input;
    die unless $registers =~ /Register A: (\d+)\nRegister B: (\d+)\nRegister C: (\d+)/;
    my ($rega, $regb, $regc) = (int $1, int $2, int $3);
    my @instructions = map int, split ',', (map s/\n//gr, split ': ', $program)[1];
    my %instruction_map = (
        0 => 'adv',
        1 => 'bxl',
        2 => 'bst',
        3 => 'jnz',
        4 => 'bxc',
        5 => 'out',
        6 => 'bdv',
        7 => 'cdv',
    );
    sub combo_getter {
        my ($state, $lit) = @_;
        die "invalid combo literal: $lit" if $lit < 0 || $lit > 6;
        return $lit if $lit < 4;
        return $state->{(qw/a b c/)[$lit - 4]};
    }
    my %handlers = (
        adv => sub {
            my ($state, $lit) = @_;
            $state->{a} = int ($state->{a} >> (combo_getter($state, $lit)));
        },
        bxl => sub {
            my ($state, $lit) = @_;
            $state->{b} ^= int $lit;
        },
        bst => sub {
            my ($state, $lit) = @_;
            $state->{b} = combo_getter($state, $lit) % 8;
        },
        jnz => sub {
            my ($state, $lit) = @_;
            if ($state->{a} != 0) {
                $state->{ip} = $lit - 2;
            }
        },
        bxc => sub {
            my ($state, $lit) = @_;
            $state->{b} ^= int $state->{c};
        },
        out => sub {
            my ($state, $lit) = @_;
            push @{$state->{output}}, combo_getter($state, $lit) % 8;
        },
        bdv => sub {
            my ($state, $lit) = @_;
            $state->{b} = int ($state->{a} >> (combo_getter($state, $lit)));
        },
        cdv => sub {
            my ($state, $lit) = @_;
            $state->{c} = int ($state->{a} >> (combo_getter($state, $lit)));
        },
    );
    sub step_state {
        my ($state) = @_;
        # die "invalid ip: $state->{ip}" if $state->{ip} < 0 || $state->{ip} >= @{$state->{program}} - 1;
        my $instruction = $instruction_map{$state->{program}[$state->{ip}]};
        my $lit = $state->{program}[$state->{ip}+1];
        # say "debug: $instruction: $lit ($state->{a}, $state->{b}, $state->{c}, @ $state->{ip})";
        # say $handlers{$instruction};
        $handlers{$instruction}->($state, $state->{program}[$state->{ip}+1]);
        $state->{ip} += 2;
    }
    # my @parsed_instructions = map [ $instruction_map{$instructions[$_]}, $instructions[$_+1] ], grep $_ % 2 == 0, 0 .. $#instructions;

    my $state = {
        a => $rega,
        b => $regb,
        c => $regc,
        ip => 0,
        program => \@instructions,
        output => [],
        step => 0,
    };
    while ($state->{ip} < @instructions) {
        last if $state->{step}++ > 1000;
        step_state($state);
    }
    return join ',', @{$state->{output}};
}
timer sub {
    # say Dumper process_day17_part1($practice_input);
    say Dumper process_day17_part1(qq/Register A: 0
Register B: 2024
Register C: 43690

Program: 4,0,5,5/);
    # say Dumper process_day17_part1($practice_input2);
    confirm process_day17_part1(qq/Register A: 0
Register B: 0
Register C: 9

Program: 2,6,5,5/), '1';
    confirm process_day17_part1(qq/Register A: 10
Register B: 0
Register C: 0

Program: 5,0,5,1,5,4/), '0,1,2';
    confirm process_day17_part1(qq/Register A: 2024
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0/), '4,2,5,6,7,7,7,7,3,1,0';

    confirm process_day17_part1(qq/Register A: 0
Register B: 29
Register C: 0

Program: 1,7,5,5/), '2';

    confirm process_day17_part1(qq/
Register A: 117440
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0/), '0,3,5,4,3,0';

    confirm process_day17_part1($practice_input), '4,6,3,5,6,3,5,2,1,0';
    # confirm process_day17_part1($practice_input2), 11048;
}

$VAR1 = '2';

confirmed: 1
confirmed: 0,1,2
confirmed: 4,2,5,6,7,7,7,7,3,1,0
confirmed: 2
confirmed: 0,3,5,4,3,0
confirmed: 4,6,3,5,6,3,5,2,1,0
[time] 0 seconds


0

Warning: Subroutine combo_getter redefined at reply input line 17.

Variable "%instruction_map" will not stay shared at reply input line 62.

Variable "%handlers" will not stay shared at reply input line 66.

Subroutine step_state redefined at reply input line 59.

Subroutine process_day17_part1 redefined at reply input line 1.


In [214]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/17/input');
    $res = process_day17_part1($input);
    say Dumper $res;
    confirm $res, '1,0,2,0,5,7,2,1,3';
};

$VAR1 = '1,0,2,0,5,7,2,1,3';

confirmed: 1,0,2,0,5,7,2,1,3
[time] 0 seconds


0

In [73]:
$res

1,0,2,0,5,7,2,1,3

In [74]:
say post_answer('2024/day/17/answer', 1, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 17 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1

In [232]:
sub process_day17_part2 {
    my ($input) = @_;
    my ($registers, $program) = split "\n\n", $input;
    my $program_code = (map s/\n//gr, split ': ', $program)[1];

    sub search_chain2 {
        my ($program, $target) = @_;
        my @target_nums = split ',', $target;
        shift @target_nums;
        if (@target_nums == 0) {
            return grep process_day17_part1(qq/Register A: $_
Register B: 0
Register C: 0

Program: $program/) eq $target, 0 .. 7;
        } else {
            return
                grep process_day17_part1(qq/Register A: $_
Register B: 0
Register C: 0

Program: $program/) eq $target,
                map { my $acc = $_; map { ($acc << 3)+$_ } 0 .. 7 }
                search_chain2($program, join ',', @target_nums);
        }
    }
    my @answers = map int, search_chain2($program_code, $program_code);
    my $smallest = (sort @answers)[0];
    # say Dumper @answers;
    # say Dumper $smallest;
    
    return $smallest;
}
timer sub {
    confirm process_day17_part2(qq/Register A: 2024
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0/), 117440;
}

confirmed: 117440
[time] 0 seconds


0

Warning: Subroutine search_chain2 redefined at reply input line 6.

Subroutine process_day17_part2 redefined at reply input line 1.


In [233]:
my $res;
timer sub {
    # 200000 in 2s
    # 1000000 in 7s
    my $input = get_challenge('2024/day/17/input');
    $res = process_day17_part2($input);
    say Dumper $res;
    confirm $res, 265652340990875;
};

$VAR1 = '265652340990875';

confirmed: 265652340990875
[time] 1 seconds


1

In [57]:
sub day17_prog {
    my @output;
    my ($a) = @_;
    while ($a > 0) {
        my $b = 0;
        my $c = 0;
    	# debug: bst: 4 (64012472, 0, 0, @ 0)
    	$b = ($a & 0x7) ^ 0x7;
    	# # debug: bxl: 7 (64012472, 0, 0, @ 2)
    	# $b ^= 7;
    	# # debug: cdv: 5 (64012472, 7, 0, @ 4)
    	# $c = $a >> $b;
    	# # debug: adv: 3 (64012472, 7, 500097, @ 6)
    	# $a = $a >> 3;
    	# # debug: bxl: 7 (8001559, 7, 500097, @ 8)
    	# $b ^= 7;
    	# # debug: bxc: 1 (8001559, 0, 500097, @ 10)
    	# $b ^= $c;
    	# debug: out: 5 (8001559, 500097, 500097, @ 12)
    	push @output, ($b ^ 7 ^ ($a >> $b)) & 0x7;
    	# debug: jnz: 0 (8001559, 500097, 500097, @ 14)
    	$a = $a >> 3;
    }
    return join ',', @output;
}

# say Dumper day17_prog(64012472);
confirm day17_prog(64012472), '1,0,2,0,5,7,2,1,3';
confirm day17_prog(63203070), '1,0,2,0,5,7,2,1,3';

sub day17_prog_searcher {
    my ($search, $start, $end) = @_;
    my @instructions = map $_ ^ 7, map int, split ',', $search;
    foreach my $i ($start .. $end) {
        # my @output;
        my $a = $i;
        my $d = 0;
        my $bad = 0;
        while ($a > 0) {
        	# debug: bst: 4 (64012472, 0, 0, @ 0)
        	my $b = ($a & 0x7) ^ 0x7;
        	# # debug: bxl: 7 (64012472, 0, 0, @ 2)
        	# $b ^= 7;
        	# # debug: cdv: 5 (64012472, 7, 0, @ 4)
        	# $c = $a >> $b;
        	# # debug: adv: 3 (64012472, 7, 500097, @ 6)
        	# $a = $a >> 3;
        	# # debug: bxl: 7 (8001559, 7, 500097, @ 8)
        	# $b ^= 7;
        	# # debug: bxc: 1 (8001559, 0, 500097, @ 10)
        	# $b ^= $c;
        	# debug: out: 5 (8001559, 500097, 500097, @ 12)
        	# push @output, ($b ^ ($a >> $b)) & 0x7;
            if ($instructions[$d++] != (($b ^ ($a >> $b)) & 0x7)) {
                $bad = 1;
                last;
            }
        	# debug: jnz: 0 (8001559, 500097, 500097, @ 14)
        	$a = $a >> 3;
        }
        # if (@output == @instructions) {
        #     my $res = join ',', @output;
        #     if ($res eq join ',', @instructions) {
        #         return $i;
        #     }
        # }
        if ($d == @instructions and not $bad) {
            # my $res = join ',', @output;
            # if ($res eq join ',', @instructions) {
                return $i;
            # }
        }
    }
}

timer sub {
    say Dumper day17_prog_searcher('1,0,2,0,5,7,2,1,3', 0, 100000000);
}

confirmed: 1,0,2,0,5,7,2,1,3
confirmed: 1,0,2,0,5,7,2,1,3
$VAR1 = 63203070;

[time] 15 seconds


15

Warning: Subroutine day17_prog redefined at reply input line 1.

Subroutine day17_prog_searcher redefined at reply input line 31.


In [204]:
say Dumper day17_prog(265652340990875);


$VAR1 = '2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0';



1

In [181]:
# 2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0
my $a = 0;
my @l = (0x7, 0x4, 0x6, 0x1, 0x6, 0x3, 0x0, 0x0, 0x3, 0x0, 0x6);
foreach my $i (0 .. $#l) {
    $a <<= 3;
    $a |= $l[$i];
}
# my $a = 0x3c;
my $b;
while ($a > 0) {
    $b = ($a & 0x7) ^ 0x7;
    say (($b ^ 7 ^ ($a >> $b)) & 0x7);
    $a >>= 3;
}

4
5
0
3
1
7
4
1
5
5
3
0


In [222]:
# 2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0

sub search_chain2 {
    my ($program, $target) = @_;
    my @target_nums = split ',', $target;
    shift @target_nums;
    if (@target_nums == 0) {
        return grep process_day17_part1(qq/Register A: $_
Register B: 0
Register C: 0

Program: $program/) eq $target, 0 .. 7;
    } else {
        return
            grep process_day17_part1(qq/Register A: $_
Register B: 0
Register C: 0

Program: $program/) eq $target,
            map { my $acc = $_; map { ($acc << 3)+$_ } 0 .. 7 }
            search_chain2($program, join ',', @target_nums);
    }
}
timer sub {
    my @answers = map int, search_chain2('2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0', '2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0');
    my $smallest = (sort @answers)[0];
    # say Dumper @answers;
    say Dumper $smallest;
}

$VAR1 = '265652340990875';

[time] 0 seconds


0

Warning: Subroutine search_chain2 redefined at reply input line 3.


In [211]:
# 2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0

sub search_chain {
    my ($target) = @_;
    my @target_nums = split ',', $target;
    shift @target_nums;
    if (@target_nums == 0) {
        return grep day17_prog($_) eq $target, 0 .. 7;
    } else {
        return grep day17_prog($_) eq $target, map { my $acc = $_; map { ($acc << 3)+$_ } 0 .. 7 } search_chain(join ',', @target_nums);
    }
}
say Dumper search_chain('2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0');


$VAR1 = '265652340990875';
$VAR2 = '265652340990877';



1

Warning: Subroutine search_chain redefined at reply input line 3.
